In [4]:
import numpy as np
import scipy.stats

The data is entered by hand from the book.

In [5]:
data = np.array((
    3.23,
    -2.50,
    1.88,
    -0.68,
    4.43,
    0.17,
    1.03,
    -0.07,
    -0.01,
    0.76,
    1.76,
    3.18,
    0.33,
    -0.31,
    0.30,
    -0.61,
    1.52,
    5.43,
    1.54,
    2.28,
    0.42,
    2.33,
    -1.03,
    4.00,
    0.39
))

First we perform the MLE computations and use the delta method to estimate the standard error.

In [292]:
sample_mean = data.mean()
population_variance = data.var()

tau_estimate = sample_mean + np.sqrt(population_variance)*scipy.stats.norm.ppf(0.95)

n = len(data)
se_delta_estimate = np.sqrt((population_variance/n)*(1 + ((scipy.stats.norm.ppf(0.95))**2)/2))

print(
    f"The sample mean is {sample_mean:.2f}.\n"
    f"The population variance is {population_variance:.2f}.\n"
    f"The estimate of tau is {tau_estimate:.2f}.\n"
    f"Using the delta method, the standard error estimate is {se_delta_estimate:.3f}."
)

The sample mean is 1.19.
The population variance is 3.30.
The estimate of tau is 4.18.
Using the delta method, the standard error estimate is 0.558.


We now perform the bootstrap estimation of the variance.

In [276]:
def parameter_estimator(data):
    return (
        data.mean(),
        data.var()
    )
    
def tau(parameter):
    mean, variance = parameter
    return mean + np.sqrt(variance)*scipy.stats.norm.ppf(0.95)

def bootstrap_resample(parameter_estimate, n):
    mean_estimate, variance_estimate = parameter_estimate
    return scipy.stats.norm.rvs(loc=mean_estimate, scale=np.sqrt(variance_estimate), size=n)

def bootstrap_replications(estimator, parameter_estimator, parameter_estimate, n, B):
    return [
        tau(parameter_estimator(
            bootstrap_resample(parameter_estimate=parameter_estimate, n=n)
        ))
    for _ in range(B)]

def se_boostrap(data, estimator, parameter_estimator, B):
    
    n = len(data)
    
    parameter_estimate = parameter_estimator(data)

    replicated_data = bootstrap_replications(
        estimator=estimator,
        parameter_estimator=parameter_estimator,
        parameter_estimate=parameter_estimate,
        n=n,
        B=B
    )
    se_boot_estimate = np.std(replicated_data)

    return se_boot_estimate

In [299]:
se_boot_estimate = se_boostrap(data, estimator=tau, parameter_estimator=parameter_estimator, B=int(1e5))

print(
    f"Using the parametric bootstrap method, the standard error estimate is {se_boot_estimate:.3f}."
)

Using the parametric bootstrap method, the standard error estimate is 0.554.
